In [3]:
import cv2
from ultralytics import YOLO
import numpy as np
from playsound import playsound
import time
import os
import threading

# ==============음성 재생을 위한 설정====================
# 1. YOLO 라벨과 실제 MP3 파일 경로를 연결하는 딕셔너리 생성
voice_files = {
    'car': 'voices/car.mp3',
    'bus': 'voices/bus.mp3',
    'truck': 'voices/truck.mp3',
    'person': 'voices/person.mp3',
    'bicycle': 'voices/bicy.mp3',        # 파일 이름에 맞게 수정
    'traffic light': 'voices/traffic.mp3' # 파일 이름에 맞게 수정
}

# 2. 음성 파일 재생 함수 및 스레드 호출 함수
def play_voice(file_path):
    try:
        if os.path.exists(file_path):
            playsound(file_path)
        else:
            print(f"경고: {file_path} 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"playsound error: {e}")

def play_voice_multi_thread(file_path):
    threading.Thread(target=play_voice, args=(file_path,), daemon=True).start()

# 3. 경고 중복 방지를 위한 변수 초기화
last_alert_time = 0
alert_interval = 5 
# =======================================================

# ==============MiDaS 설정 (이전과 동일) ====================
midas_model = cv2.dnn.readNet("model-small.onnx")

def estimate_depth(frame):
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (256,256), swapRB=True, crop=False)
    midas_model.setInput(blob)
    output = midas_model.forward()
    depth_map = output[0,:,:]
    return cv2.resize(
        cv2.normalize(depth_map, None, 0, 1, cv2.NORM_MINMAX), (frame.shape[1], frame.shape[0]))
# =============================================

# YOLO가 인식하는 위험 객체 리스트 (이전과 동일)
Danger_classes = ['car', 'bus', 'truck', 'person', 'bicycle', 'traffic light']

cap = cv2.VideoCapture('SeoulWalk.mp4')
yolo_model = YOLO('yolov8n.pt')
if not cap.isOpened():
    print("비디오 열기 실패")
    exit()

frame_idx = 0
depth_map = None 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % 5 == 0 or depth_map is None:
        depth_map = estimate_depth(frame)
    
    results = yolo_model(frame, conf=0.5, verbose=False)[0]
    
    closest_object_label = None
    highest_depth_score = 0.6 

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = yolo_model.names[cls_id]
        
        if label in Danger_classes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
            
            if 0 <= cy < depth_map.shape[0] and 0 <= cx < depth_map.shape[1]:
                depth_score = depth_map[cy, cx]
                if depth_score > highest_depth_score:
                    highest_depth_score = depth_score
                    closest_object_label = label
                
                cv2.putText(frame, f"{label} detected!", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                if depth_score > 0.6:
                    cv2.putText(frame, "near!", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # ==============음성 출력 (수정된 로직) ======================
    now = time.time()
    if closest_object_label and (now - last_alert_time > alert_interval):
        # 4. 딕셔너리에서 라벨에 해당하는 파일 경로를 찾아 재생
        file_to_play = voice_files.get(closest_object_label)
        if file_to_play:
            play_voice_multi_thread(file_to_play)
            last_alert_time = now
    # =============================================

    cv2.imshow("GuidePath", frame)
    frame_idx += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()